# Revision de Evaluacion de Pruebas

Buscando en diferentes articulos y Benchmarks, de LLM's, me tope con evaluaciones comunes, por ejemplo
las usadas en Human Last Exam, algunos de ellos son Framework's como HELM (Holistic Evaluation of Language Models) y otros metodos de evaluacion como MMLU (Massive Multitask Language Understanding) y BIG-Bench. En el siguiente Jupyter evaluare el desempeño de ellos con algunas preguntas que le he planteado a los modelos de lenguaje, con los cuales se hizo el estudio de ablacion de prompting.

## Explicación del código

El código implementa una **clase llamada `AutomaticEvaluator`**, cuyo propósito es **evaluar automáticamente la similitud entre dos textos**: uno considerado como **referencia** (por ejemplo, una respuesta ideal) y otro como **respuesta generada** (por ejemplo, por un modelo de lenguaje).

Esta clase calcula varias **métricas de similitud textual** ampliamente utilizadas en el procesamiento de lenguaje natural (PLN).

---

### 1. Importación de librerías

Se utilizan varias librerías de Python:

* **TfidfVectorizer**: convierte los textos en vectores numéricos basados en la frecuencia de palabras (TF-IDF), lo que permite medir similitud entre textos.
* **cosine_similarity**: calcula la similitud entre dos vectores (en este caso, los textos transformados con TF-IDF).
* **rouge_scorer**: calcula las métricas **ROUGE**, muy usadas para evaluar la calidad de resúmenes automáticos y respuestas generadas.
* **numpy**: permite realizar operaciones numéricas, como calcular promedios.

---

### 2. Definición de la clase *AutomaticEvaluator*

La clase `AutomaticEvaluator` inicializa un evaluador automático que usa el paquete `rouge_scorer` para calcular tres métricas ROUGE:

* **ROUGE-1**: mide la superposición de unigramas (palabras individuales).
* **ROUGE-2**: mide la superposición de bigramas (pares consecutivos de palabras).
* **ROUGE-L**: mide la longitud de la subsecuencia común más larga entre los textos.

Además, se activa la opción *use_stemmer=True* para comparar las raíces de las palabras, de modo que “running” y “run” se consideren equivalentes.

---

### 3. Cálculo de métricas de similitud

El método principal, `calculate_similarity_metrics`, recibe dos cadenas de texto:

* **reference**: el texto de referencia o respuesta esperada.
* **response**: el texto generado que se desea evaluar.

El método devuelve un conjunto de métricas que cuantifican la similitud entre ambos textos.

#### a) Métricas ROUGE

Se calculan las métricas **ROUGE-1**, **ROUGE-2** y **ROUGE-L**, obteniendo valores de precisión, recall y F-measure.
En este caso, se utiliza únicamente el **F-measure**, que representa un equilibrio entre precisión y exhaustividad.

#### b) Similitud del coseno (TF-IDF)

Los textos se transforman en vectores mediante TF-IDF, y luego se calcula la **similitud del coseno**, que mide cuán alineados están esos vectores.

El valor de la similitud del coseno varía entre:

* **1**: textos idénticos o muy similares.
* **0**: textos completamente diferentes.

Esta métrica captura similitud semántica basada en la distribución de las palabras.

#### c) Superposición de palabras

Se calcula el porcentaje de **palabras compartidas** entre el texto de referencia y la respuesta, sin distinguir mayúsculas o minúsculas.
Esta métrica simple ofrece una medida intuitiva del grado de coincidencia léxica.

---

### 4. Retorno de resultados

El método devuelve un **diccionario de métricas**, que incluye:

* ROUGE-1
* ROUGE-2
* ROUGE-L
* Similitud del coseno (TF-IDF)
* Superposición de palabras
* Un **promedio global** entre las métricas ROUGE y la similitud del coseno

Este valor promedio proporciona una evaluación general de la similitud textual entre la referencia y la respuesta generada.


In [6]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from rouge_score import rouge_scorer
import numpy as np

class AutomaticEvaluator:
    def __init__(self):
        self.scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

    def calculate_similarity_metrics(self, reference: str, response: str) -> dict:
        """Calcula métricas automáticas de similitud"""

        # ROUGE scores
        rouge_scores = self.scorer.score(reference, response)

        # Similitud coseno con TF-IDF
        vectorizer = TfidfVectorizer().fit_transform([reference, response])
        cosine_sim = cosine_similarity(vectorizer[0:1], vectorizer[1:2])[0][0]

        # Similitud por superposición de palabras
        ref_words = set(reference.lower().split())
        resp_words = set(response.lower().split())

        if len(ref_words) > 0:
            word_overlap = len(ref_words.intersection(resp_words)) / len(ref_words)
        else:
            word_overlap = 0

        return {
            'rouge1': round(float(rouge_scores['rouge1'].fmeasure),4),
            'rouge2': round(float(rouge_scores['rouge2'].fmeasure),4),
            'rougeL': round(float(rouge_scores['rougeL'].fmeasure),4),
            'cosine_similarity': round(float(cosine_sim),4),
            'word_overlap': round(float(word_overlap),4),
            'score_promedio': round(float(np.mean([
                rouge_scores['rouge1'].fmeasure,
                rouge_scores['rouge2'].fmeasure,
                rouge_scores['rougeL'].fmeasure,
                cosine_sim
            ])),4)
        }
if __name__ == "__main__":
    # USO
    auto_eval = AutomaticEvaluator()
    metrics = auto_eval.calculate_similarity_metrics(
        reference="""Lo mas recomendable es coger la linea amarilla en sentido de ida
     Las estaciones que cruzarás son: AA1SC → AB2SC → AC3SC → AD4RF → AE5VE → AF6SC → AG7BH""",
        response="""Puedes llegar desde la estación AA1SC a la estación AG7BH tomando la línea amarilla. Primero, dirígete hacia la estación AA1SC, que es la primera estación del sentido ida de la línea amarilla. Luego, sigue el recorrido de la línea amarilla en sentido ida: AA1SC, AB2SC, AC3SC, AD4RF, AE5VE, AF6SC y finalmente AG7BH."""
    )
    print(metrics)


{'rouge1': 0.3529, 'rouge2': 0.1687, 'rougeL': 0.3059, 'cosine_similarity': 0.3908, 'word_overlap': 0.44, 'score_promedio': 0.3046}
